# Import librarly

In [1]:
import json
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import math
import numpy as np

# Read dataset

In [2]:
# Do not separate by a comma if there is no space after the comma
# If you use a regular expression as a delimiter, you need to specify the Python engine
twitterData = pd.read_csv("data.csv", sep=',(?=\S)', engine='python')

twitterData.head()

,id,screenName,tags,avatar,followersCount,friendsCount,lang,lastSeen,tweetId,friends
0,"""1969527638""","""LlngoMakeEmCum_""","[ ""#nationaldogday"" ]","""http://pbs.twimg.com/profile_images/534286217...",319,112,"""en""",1472271687519,"""769310701580083200""","[ ""1969574754"", ""1969295556"", ""1969284056"", ""1..."
1,"""51878493""","""_notmichelle""","[ ""#nationaldogday"" ]","""http://pbs.twimg.com/profile_images/761977602...",275,115,"""en""",1472270622663,"""769309490038439936""","[ ""60789485"", ""2420931980"", ""2899776756"", ""127..."
2,"""1393409100""","""jesseayye""","[ ""#narcos"" ]","""http://pbs.twimg.com/profile_images/713282938...",120,107,"""en""",1472804144409,"""771622623583186944""","[ ""86868062"", ""19697415"", ""2998836604"", ""45629..."
3,"""232891415""","""MrBrianLloyd""","[ ""#gloryoutnow"" ]","""http://pbs.twimg.com/profile_images/133440668...",492,325,"""en""",1472269186776,"""769308131696467969""","[ ""361335082"", ""1405248468"", ""24626354"", ""7256..."
4,"""710130422907207680""","""sarahdorat_16""","[ ""#nationaldogday"" ]","""http://pbs.twimg.com/profile_images/767180520...",128,218,"""en""",1472271397356,"""769309769144283137""","[ ""1571896093"", ""768938323612008448"", ""2548665..."


## Modify dataset

In [3]:
# 1. removing unnecessary double quotes
twitterData["id"] = twitterData["id"].apply(lambda x: x[1:-1])
    
# 2. The list of friends (people followed) is saved as a single string. To convert it to a list of friend IDs, use a JSON loader.
for column in ["tags", "friends"]:
    twitterData[column] = twitterData[column].apply(lambda x: json.loads(x))

# 3. drop unused column for this project
twitterData = twitterData.drop(["lang", 'avatar', 'tweetId', 'screenName', 'lastSeen'], axis=1)

twitterData.head()

,id,tags,followersCount,friendsCount,friends
0,1969527638,[#nationaldogday],319,112,"[1969574754, 1969295556, 1969284056, 196961221..."
1,51878493,[#nationaldogday],275,115,"[60789485, 2420931980, 2899776756, 127410795, ..."
2,1393409100,[#narcos],120,107,"[86868062, 19697415, 2998836604, 456295047, 74..."
3,232891415,[#gloryoutnow],492,325,"[361335082, 1405248468, 24626354, 725675895965..."
4,710130422907207680,[#nationaldogday],128,218,"[1571896093, 768938323612008448, 2548665930, 3..."


In [4]:
twitterData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              40000 non-null  object
 1   tags            40000 non-null  object
 2   followersCount  40000 non-null  int64 
 3   friendsCount    40000 non-null  int64 
 4   friends         40000 non-null  object
dtypes: int64(2), object(3)
memory usage: 1.5+ MB


# Prerpocess

In [5]:
# friends: the list of IDs the user follows
# friendsCount: the length of the friends list
twitterData.describe()

,followersCount,friendsCount
count,4.000000e+04,40000.000000
mean,3.692349e+03,919.101450
std,9.136179e+04,3417.849318
min,1.010000e+02,101.000000
25%,2.420000e+02,253.000000
50%,4.400000e+02,423.000000
75%,9.240000e+02,801.000000
max,1.476555e+07,238749.000000


In [6]:
# create a dictionary to find index from id
id_index_dict = {}
for index, row in twitterData.iterrows():
    id_index_dict[row['id']] = index

In [7]:
# show the number of nodes, edges and the number of communities
def print_graph_summary(G):
    print("Number of nodes: {}".format(G.number_of_nodes()))
    print("Number of edges: {}".format(G.number_of_edges()))
    # detect communities
    communities = list(nx.community.greedy_modularity_communities(G))
    print("Number of communities: {}".format(len(communities)))
    return communities

In [8]:
# create directed graph: the node is the user, the edge is the connection between users
G = nx.DiGraph()

for _, row in twitterData.iterrows():
    for friend in row['friends']:
        # ignore the users who are not in this dataset
        if friend in id_index_dict:
            G.add_edge(row['id'], friend)
            
# show the number of nodes and edges
print("Number of nodes: {}".format(G.number_of_nodes()))
print("Number of edges: {}".format(G.number_of_edges()))

Number of nodes: 33734
Number of edges: 184548


### Degree-based filtering:

In [9]:
min_in_degree = 20
filtered_nodes = [node for node, in_degree in G.in_degree() if in_degree >= min_in_degree]
G = G.subgraph(filtered_nodes)

communities = print_graph_summary(G)

Number of nodes: 1633
Number of edges: 25328
Number of communities: 30


### K-Core Decomposition:

In [ ]:
k_core = 20
G = nx.k_core(G, k=k_core)

communities = print_graph_summary(G)

# Network Characterization

In [ ]:
# print number of nodes and edges with format
print("Number of nodes: {}\n".format(G.number_of_nodes()))
print("Number of edges: {}\n".format(G.number_of_edges()))

In [ ]:
# find the five nodes with the highest clustering coefficients with format
hi_clustring_coeff = sorted(nx.clustering(G).items(), key=lambda x: x[1], reverse=True)[:5]

# find the five nodes with the highest betweenness centrality with format
hi_betweeness_cen = sorted(nx.betweenness_centrality(G).items(), key=lambda x: x[1], reverse=True)[:5]

In [ ]:
# enumerate the five nodes with the highest clustering coefficients
for i, (node, coeff) in enumerate(hi_clustring_coeff):
    # get index from id
    index = id_index_dict[node]
    print("{}. clustering coefficient: {}".format(i+1, coeff))
    # print the row of the node
    print(twitterData.iloc[index])
    print()

In [ ]:
# enumerate the five nodes with the highest betweenness centrality
for i, (node, betweenness) in enumerate(hi_betweeness_cen):
    # get index from id
    index = id_index_dict[node]
    print("{}. betweenness centrality: {}".format(i+1, betweenness))
    # print the row of the node
    print(twitterData.iloc[index])
    print()

In [ ]:
# print number of connected components with format
print("Number of connected components: {}".format(nx.number_connected_components(G.to_undirected())))
# print the diameter of the graph with format
for i, component in enumerate(nx.connected_components(G.to_undirected())):
    print("Diameter of the component {}: {}".format(i + 1, nx.diameter(G.subgraph(component).to_undirected())))

### Visualization

In [ ]:
# combine the nodes in communities
community_dict = {node: -1 for node in G.nodes()}
community_dict = {node: cid for cid, community in enumerate(communities) for node in community}

# set the color of nodes
colors = list(mcolors.TABLEAU_COLORS.keys()) + list(mcolors.CSS4_COLORS.keys())
node_color = [colors[community_dict[node] % len(colors)] for node in G.nodes()]

In [ ]:
nx.draw(G, node_color=node_color, with_labels=False, node_size=10, alpha=0.5, width=0.1)
plt.show()

In [ ]:
# calculate the number of rows and columns of subplots
n_communities = len(communities)
n_columns = math.ceil(math.sqrt(n_communities))
n_rows = math.ceil(n_communities / n_columns)

# set the size of subplots
fig, axes = plt.subplots(nrows=n_rows, ncols=n_columns, figsize=(3 * n_columns, 3 * n_rows))

# draw subgraphs
for i, community in enumerate(communities):
    subgraph = G.subgraph(community)

    # set the color
    subgraph_node_color = [colors[community_dict[node] % len(colors)] for node in subgraph.nodes()]

    ax = axes[i // n_columns, i % n_columns]
    ax.set_title(f"Community {i + 1}")
    nx.draw(subgraph, node_color=subgraph_node_color, with_labels=False, node_size=10, alpha=0.5, width=0.1, ax=ax)

# delete empty subplots
for i in range(n_communities, n_rows * n_columns):
    fig.delaxes(axes[i // n_columns, i % n_columns])

# show the graphs
plt.tight_layout()
plt.show()


In [ ]:
# create a list of graph of all communities
graphs = [G.subgraph(community) for community in communities]

# community_tags: the list of sets of tags in each community
# community_ratios: the list of ratios of tags in each community
community_tags = []
community_ratios = []

for i, graph in enumerate(graphs):
    # print the summary of the graph
    print(f"Community {i + 1}")
    print("\tNumber of nodes: {}".format(graph.number_of_nodes()))
    print("\tNumber of edges: {}".format(graph.number_of_edges()))

    # using id_index_dict, get the index of nodes in the graph
    # then, get the tags of the nodes
    # finally, get the set of tags in the graph
    tags = set()
    for node in graph.nodes():
        tags.update(twitterData['tags'][id_index_dict[node]])
        
    # count the number of tags in the graph
    tag_count = {tag: 0 for tag in tags}
    total_count = 0
    for node in graph.nodes():
        for tag in twitterData['tags'][id_index_dict[node]]:
            tag_count[tag] += 1
            total_count += 1

    # sort the tags by the number of appearance
    tag_count = sorted(tag_count.items(), key=lambda x: x[1], reverse=True)
    
    # display the top 3 tags and their ratios
    top_tags = []
    top_ratios = []
    for tag, count in tag_count[:3]:
        ratio = count / total_count * 100
        print(f"\t- {tag}: {ratio:.2f}%")
        top_tags.append(tag)
        top_ratios.append(ratio)

    if len(top_tags) < 3:
        top_tags = top_tags + ["", ""]
        top_ratios = top_ratios + [0, 0]
    community_tags.append(top_tags)
    community_ratios.append(top_ratios)

    print()

In [ ]:
# visualization
n_communities = len(communities)
bar_width = 0.3
index = np.arange(3)

fig, axes = plt.subplots(nrows=n_rows, ncols=n_columns, figsize=(4 * n_columns, 4 * n_rows))

for i in range(n_communities):
    ax = axes[i // n_columns, i % n_columns]
    ax.bar(index - bar_width / 2, community_ratios[i], width=bar_width, tick_label=community_tags[i])
    ax.set_title(f"Community {i + 1}")
    ax.set_ylim(0, 110)  #  fix the range of y-axis
    #ax.set_xlabel('Hashtags')
    ax.set_ylabel('Occurrence Ratio (%)')
    # rotate the labels of x-axis for readability
    ax.tick_params(axis='x', rotation=45)

# delete empty subplots
for i in range(n_communities, n_rows * n_columns):
    fig.delaxes(axes[i // n_columns, i % n_columns])

plt.tight_layout()
plt.show()